# Create Heat Map for significant Mismatch Repair genes

This notebook looks at the significant genes in at least one cancer in the Mismatch Repair pathway. Pancancer heat maps are created with circle size showing significance and color showing differences in median.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
import gseapy as gp
import re
import sys 

import cptac
import cptac.utils as u

import plot_utils as pu

# Step 1: Run GSEA for significant genes in at least 1 cancer

First read in sig_pval_all_proteins.csv into a df. This csv file contains only genes with a significant p-value in at least one cancer. Then run GSEA using a list of genes from the df.

In [2]:
root = R'~\Github\WhenMutationsDontMatter\PTEN\Step_3_trans_effect\csv'
sig_df = pd.read_csv(root+R'\sig_pval_heatmap.csv')

prot_list = list(sig_df.Proteomics) # list of genes with a sig pval in >= 1 cancer
prot_enr = gp.enrichr(gene_list = prot_list, description='Tumor_partition', gene_sets='KEGG_2016', 
                       outdir='/Enrichr')
prot_enr.res2d.head()

,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes,Gene_set
0,Spliceosome Homo sapiens hsa03040,88/134,2.936551e-44,8.604095e-42,0,0,4.860965,487.246193,RBM25;EIF4A3;DDX42;HNRNPU;PRPF19;USP39;EFTUD2;...,KEGG_2016
1,RNA transport Homo sapiens hsa03013,77/172,1.113872e-23,1.631823e-21,0,0,3.313652,175.131871,CYFIP2;NUP107;NUP188;RPP30;EIF4A3;PHAX;PNN;XPO...,KEGG_2016
2,DNA replication Homo sapiens hsa03030,23/36,3.613010e-12,3.528707e-10,0,0,4.729007,124.592696,RFC5;FEN1;RFC3;PCNA;RFC4;MCM7;RFC1;RFC2;PRIM1;...,KEGG_2016
3,mRNA surveillance pathway Homo sapiens hsa03015,38/91,2.801846e-11,2.052353e-09,0,0,3.090914,75.103503,SMG1;RBM8A;RNMT;CSTF3;EIF4A3;MSI1;PPP2R2A;MSI2...,KEGG_2016
4,Mismatch repair Homo sapiens hsa03430,15/23,1.473843e-08,8.636717e-07,0,0,4.827342,87.050531,RFC5;RFC3;PCNA;RFC4;RFC1;RFC2;RPA1;MLH1;POLD3;...,KEGG_2016


# Step 2: Get the list of significant genes in the DNA Replication pathway

In [3]:
mis_repair = prot_enr.res2d.Genes[4]
genes = mis_repair.split(';')
print('total genes:',len(genes))

total genes: 15


# Step 3: Create HeatMap

Slice out the significant genes in the pathway from sig_df (sig_pval_heatmap.csv).

In [11]:
bool_df = sig_df.Proteomics.isin(genes)
plot_df = sig_df[bool_df]
plot_df.head()

,Proteomics,P_Value,Medians,Cancer
131,PCNA,0.000019,0.631486,Gbm
133,POLD2,0.000020,0.405249,Gbm
178,POLD3,0.000036,0.483613,Gbm
179,RFC5,0.000037,0.481635,Gbm
346,RFC3,0.000151,0.443671,Gbm


In [12]:
# log p-vals for right scale in plot (bigger circle, smaller pval)
plot_df['size'] = plot_df['P_Value'].apply(lambda x: -1*(np.log(x)/10))
plot_df

C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Proteomics,P_Value,Medians,Cancer,size
131,PCNA,0.000019,0.631486,Gbm,1.087410
133,POLD2,0.000020,0.405249,Gbm,1.081765
178,POLD3,0.000036,0.483613,Gbm,1.022357
179,RFC5,0.000037,0.481635,Gbm,1.021302
346,RFC3,0.000151,0.443671,Gbm,0.879685
...,...,...,...,...,...
18591,MLH1,0.653830,0.079600,Colon,0.042491
18832,MSH6,0.770655,0.032000,Colon,0.026051
19026,MSH2,0.865216,-0.010000,Colon,0.014478
19145,RFC4,0.921316,-0.104850,Colon,0.008195


In [13]:
pu.plotCircleHeatMap(plot_df, circle_var = 'size', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                     graph_width=1000)

C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\plot_utils\__init__.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['size'] = np.where(df[circle_var]<0, np.abs(df[circle_var]), df[circle_var])*50


Loading BokehJS ...

# Step 4: Create a HeatMap with both pos and neg differences in median

 Read in the pos_neg_df.csv to create a df with only genes that have a pos and neg difference in median in different cancers. Slice out the genes that have a pos and neg difference in median in the pathway using the list of genes with a significant p-value in the pathway. 

In [16]:
pos_neg_df = pd.read_csv(root+R'\pos_neg_df.csv')

In [24]:
get = pos_neg_df.Proteomics.isin(genes) # bool df where True has both pos and neg
genes_pn = pos_neg_df[get] # Keep only genes with pos and neg
genes_pn.Proteomics.unique()

array(['RFC5', 'RFC2', 'RFC4', 'SSBP1', 'MSH2', 'MSH6'], dtype=object)

In [25]:
# log p-vals for right scale in plot (bigger circle, smaller pval)
genes_pn['size'] = genes_pn['P_Value'].apply(lambda x: -1*(np.log(x)/10))

C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [26]:
pu.plotCircleHeatMap(genes_pn, circle_var = 'size', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                    graph_width=700)

Loading BokehJS ...